## Crypto data downloader

In [4]:
starting_date = '2017-01-01T00:00:00Z'
ending_date = '2022-04-25T00:00:00Z'
pair = 'BTC/USDT'
tf = "1d"
exchange = 'Binance'

to_present = True
pair_id = pair.split('/')[0] + "-" + pair.split('/')[-1]
print("Pair : " + pair_id)

Pair : BTC-USDT


In [5]:
import ccxt
from datetime import datetime as dt
from freqtrade.data.converter import ohlcv_to_dataframe
import pandas as pd
import time

exchange_dict = {'Binance':ccxt.binance(),
                'Kucoin':ccxt.kucoin(),
                'Coinex':ccxt.coinex(),
                'Kraken':ccxt.kraken(),
                'Coinbase':ccxt.coinbase()}
ex = exchange_dict[exchange]

print(ex.iso8601(ex.milliseconds()), 'Loading markets')
#markets = ex.load_markets()
print(ex.iso8601(ex.milliseconds()), 'Markets loaded')

from_timestamp = ex.parse8601(starting_date)
if to_present:
    now = ex.milliseconds()
else:
    now = ex.parse8601(ending_date)
df_list = []
while from_timestamp < now:
    print('Fetching candles starting from', ex.iso8601(from_timestamp))
    try:
        ohlcvs = ex.fetch_ohlcv(pair, tf, from_timestamp)
    except:
        print("Retrying...")
        continue
    time.sleep(ex.rateLimit / 1000)
    if not len(ohlcvs):
        break
    from_timestamp = ohlcvs[-1][0] + ex.parse_timeframe(tf) * 1000
    df_list.append(ohlcv_to_dataframe(ohlcvs, tf, pair=pair, drop_incomplete=False))

2022-04-27T15:19:05.080Z Loading markets
2022-04-27T15:19:05.080Z Markets loaded
Fetching candles starting from 2017-01-01T00:00:00.000Z
Fetching candles starting from 2018-12-30T00:00:00.000Z
Fetching candles starting from 2020-05-13T00:00:00.000Z
Fetching candles starting from 2021-09-25T00:00:00.000Z


In [6]:
df = pd.concat(df_list, axis=0)
df.reset_index(inplace=True)
df.drop('index', axis=1, inplace=True)
print('Starting date :', df['date'].iloc[0])
print('Ending date :', df['date'].iloc[-1])
path = f"{pair_id}_{tf}.csv"
df.to_csv(path, index=False)
print(f'--> Saved to {path} .')

Starting date : 2017-08-17 00:00:00+00:00
Ending date : 2022-04-27 00:00:00+00:00
--> Saved to BTC-USDT_1d.csv .


In [7]:
df_csv = pd.read_csv(path)
print('DF size: ', df_csv.shape[0], 'rows')
df_csv.head()

DF size:  1715 rows


,date,open,high,low,close,volume
0,2017-08-17 00:00:00+00:00,4261.48,4485.39,4200.74,4285.08,795.150377
1,2017-08-18 00:00:00+00:00,4285.08,4371.52,3938.77,4108.37,1199.888264
2,2017-08-19 00:00:00+00:00,4108.37,4184.69,3850.00,4139.98,381.309763
3,2017-08-20 00:00:00+00:00,4120.98,4211.08,4032.62,4086.29,467.083022
4,2017-08-21 00:00:00+00:00,4069.13,4119.62,3911.79,4016.00,691.743060
